In [ ]:
import os
import httpx

from pydantic import BaseModel
from typing import Optional, Any

from dotenv import load_dotenv
load_dotenv()

In [ ]:
db_url = f"{os.environ.get("AG3NTS_CENTRALA_URL")}/apidb"
api_key = os.environ.get("AG3NTS_API_KEY")

In [ ]:
class ApiDbQuery(BaseModel):
    task: Optional[str] = "database"
    apikey: Optional[str] = api_key
    query: str

class ApiDbResponse(BaseModel):
    reply: Any
    error: str

def query_db(query: str):
    """Send prompt to ApiDb"""
    data = ApiDbQuery(query=query)
    response = httpx.post(db_url, data=data.model_dump_json())
    return ApiDbResponse(**response.json())

In [ ]:

# contains connection pairs as {"user1_id": id1, "user2_id": id2}
connections_db = query_db("select * from connections")

# contains users as {"id": id, "username": u, "access_level": a, "is_active": i, "lastlog": l}
users_db = query_db("select * from users")

In [ ]:
# use objects for convenience

class Connection(BaseModel):
    user1_id: int
    user2_id: int

class User(BaseModel):
    id: int
    username: str
    access_level: str
    is_active: int
    lastlog: str

connections = [
    Connection(**conn) for conn in connections_db.reply
]

users = [
    User(**user) for user in users_db.reply
]

In [ ]:
# define database settings and set imports

from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
auth = ("neo4j", "testtest")

In [ ]:
# define functions to interact with database
# those functions can be used by LLM as tools or manually to organize the data

def add_user_tx(tx, user: User):
    qry = """
    MERGE (u:User {
        id: $id,
        username: $username,
        accessLevel: $access_level,
        isActive: $is_active,
        lastlog: $lastlog
    })
    """
    return tx.run(
        qry, id=user.id, username=user.username, access_level=user.access_level,
        is_active=user.is_active, lastlog=user.lastlog
    )

def add_user_relationship(tx, user_a: int, user_b: int):
    qry = """
    MATCH (ua:User {id:$id_a})
    MATCH (ub:User {id:$id_b})
    MERGE (ua)-[r:KNOWS]->(ub)
    """
    return tx.run(qry, id_a=user_a, id_b=user_b)

In [ ]:
# add all users to database

with GraphDatabase.driver(uri, auth=auth) as driver:
    with driver.session(database="neo4j") as session:
        for user in users:
            session.execute_write(add_user_tx, user=user)

In [ ]:
# add user relationships to database

with GraphDatabase.driver(uri, auth=auth) as driver:
    with driver.session(database="neo4j") as session:
        for conn in connections:
            session.execute_write(add_user_relationship, user_a=conn.user1_id, user_b=conn.user2_id)

In [ ]:
# query the database to find the shortest path from user_a to user_b

def find_shortest_path_tx(tx, id_a: int, id_b: int):
    qry = """
    MATCH p = shortestPath((ua:User)-[:KNOWS*]->(ub:User))
    WHERE ua.id = $id_a AND ub.id = $id_b
    RETURN p AS result
    """
    result = tx.run(qry, id_a=id_a, id_b=id_b)
    return result.single()

In [ ]:
user_a = list(filter(lambda user: user.username == "Rafał", users))[0]
user_b = list(filter(lambda user: user.username == "Barbara", users))[0]

with GraphDatabase.driver(uri, auth=auth) as driver:
    with driver.session(database="neo4j") as session:
        result = session.execute_read(find_shortest_path_tx, id_a=user_a.id, id_b=user_b.id)
        print(result)

In [ ]:
answer_a = ", ".join([
    item[1]
    for node in result["result"].nodes
    for item in node.items()
    if item[0] == "username"
])

In [ ]:
answer_a

In [ ]:
from aidevs3.poligon import send

load_dotenv()

key = os.environ.get("AG3NTS_API_KEY")
url = f"{os.environ.get("AG3NTS_CENTRALA_URL")}/report"

res = send(url, answer=answer_a, apikey=key, task="connections")
print(res)